# NISQ SDP

Semidefinite programming with NISQ computer. 
Solves the Lovasz-Theta number or a Bell game using a SDP via a quantum computer.

Set model=0 for Lovasz-Theta and model=1 for Bell game.

The classical SDP solves a cost function with constraints.

For NISQ SDP, we use following approach:

We use an ansatz state (either a product state of all zero, or a random state generated via randomized circuit).
The ansatz space of size n_ansatz is generated by applying various Pauli X strings onto the ansatz state.
Then, we decompose the cost matrix and the constraints into a sum of Pauli strings.
We use the ansatz space to measure various overlaps between them usiing the Pauli strings of cost and constraint.
Then, the measured overlaps are used to solve an SDP that gives the solution.
The convergence depends on the n_ansatz parameter.



The code requires qutip as well as cvxpy.

@author: Tobias Haug (github txhaug), Kishor Bharti

Contact at thaug@ic.ac.uk


In [1]:
import qutip as qt

from functools import partial
from functools import reduce

import operator
from operator import mul

import numpy as np

import cvxpy as cp

# Parameters

1. Seed of random generator

In [2]:
seed = 1

2. Model
    - 0: Lovasz-Theta number of N=3 qubits with specified graph
    - 1: Bell game

In [3]:
model = 0

3. Type of Ansatz state
    - 0: All zero product state
    - 1: Randomized quantum circuit composed of Y rotations and CNOT gates
        <img src="randomized_quantum_circuit.png" width=600 height=300>

In [4]:
ansatz_state = 0

In [5]:
## Lovasz-Theta number for graph
if model == 0:
    n_qubits = 3
    ## graph for constraints to solve for Lovasz-Theta number
    graph = np.array([
           [0,1,0,0,1,0,0,1],
           [1,0,1,0,0,1,0,0],
           [0,1,0,1,0,0,1,0],
           [0,0,1,0,1,0,0,1],
           [1,0,0,1,0,1,0,0],
           [0,1,0,0,1,0,1,0],
           [0,0,1,0,0,1,0,1],
           [1,0,0,1,0,0,1,0]])
## Bell game
elif model == 1:
    n_qubits = 2

4. Number of ansatz states to use

    The more the ansatz states, the better the convergence

    **Upper bound**: 2**n_qubits

In [6]:
n_ansatz = 8
assert n_ansatz <= 2**n_qubits, f"Number of ansatz states cannot exceed {2**n_qubits}"

5. Verbose NSS

In [7]:
verbose_SDP = True

# Functions

In [8]:
# random generator used
rng = np.random.default_rng(seed)

In [9]:
# Compute product of all the elements in
prod = lambda factors: reduce(mul, factors, 1)

In [10]:
# Flatten a list into an array
flatten = lambda lst: [elem for sublist in lst for elem in sublist]

### Fock operators

In [11]:
# tensors operators together 
def genFockOp(op, position, size, levels=2, opdim=0):
    opList = [qt.qeye(levels) for _ in range(size-opdim)]
    opList[position] = op
    return qt.tensor(opList)

In [12]:
#operators for circuit
levels = 2
opZ = [genFockOp(qt.sigmaz(), i, n_qubits, levels) for i in range(n_qubits)]
opX = [genFockOp(qt.sigmax(), i, n_qubits, levels) for i in range(n_qubits)]
opY = [genFockOp(qt.sigmay(), i, n_qubits, levels) for i in range(n_qubits)]
opI = genFockOp(qt.qeye(levels), 0, n_qubits)

opZ

In [25]:
print(opZ)

[Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]], Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]], Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0

opX

In [26]:
print(opX)

[Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]], Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]], Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]]


opY

In [27]:
print(opY)

[Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]], Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+

opI

In [28]:
print(opI)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [13]:
# get Pauli operator from pauli string
def get_pauli_op(pauli_string, opI, opX, opY, opZ):
    """
        How does multiplication work here?
    """
    pauli_circuit = opI
    
    for i in range(len(pauli_string)):
        if pauli_string[i] != 0:
            if pauli_string[i] == 1: pauli_circuit *= opX[i]
            elif pauli_string[i] == 2: pauli_circuit *= opY[i]
            elif pauli_string[i] == 3: pauli_circuit *= opZ[i]

    return pauli_circuit

In [14]:
# Convert a number to base-b as a int list
def numberToBase(n, b, n_qubits):
    digits = np.zeros(n_qubits, dtype=int)
    if n == 0: return digits
    counter = 0
    while n:
        digits[counter] = int(n % b)
        n //= b
        counter += 1
    return digits[::-1]

In [40]:
LT_pauli_list = np.zeros([64, 3], dtype=int)
for k in range(64):
        LT_pauli_list[k,:] = numberToBase(k, 4, 3)
LT_pauli_list

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 0, 3],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 1, 3],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [0, 2, 3],
       [0, 3, 0],
       [0, 3, 1],
       [0, 3, 2],
       [0, 3, 3],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 0, 3],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 1, 3],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [1, 2, 3],
       [1, 3, 0],
       [1, 3, 1],
       [1, 3, 2],
       [1, 3, 3],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 0, 3],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 1, 3],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2],
       [2, 2, 3],
       [2, 3, 0],
       [2, 3, 1],
       [2, 3, 2],
       [2, 3, 3],
       [3, 0, 0],
       [3, 0, 1],
       [3, 0, 2],
       [3, 0, 3],
       [3, 1, 0],
       [3, 1, 1],
       [3, 1, 2],
       [3,

In [41]:
LT_weights = np.zeros(64, dtype=np.complex128)

sigma_x = np.array(
        [
            [0, 1],  
            [ 1, 0]
        ], 
        dtype=np.complex128)
sigma_y = np.array(
        [
            [0, -1j],
            [1j, 0]
        ], 
        dtype=np.complex128)
sigma_z = np.array(
        [
            [1, 0],  
            [0, -1]
        ], 
        dtype=np.complex128)
I = np.array(
        [
            [1, 0],  
            [ 0, 1]
        ], 
        dtype=np.complex128)
S = [I, sigma_x, sigma_y, sigma_z]

for k in range(64):
    LT_pauli = S[LT_pauli_list[k][0]]

    for n in range(1, 3):
        LT_pauli = np.kron(LT_pauli, S[LT_pauli_list[k][n]])

    qt_dims = [
        [2 for _ in range(3)],
        [2 for _ in range(3)]
    ]
    H = qt.Qobj(np.ones([8, 8]), dims=qt_dims)
    LT_weights[k] = 1/8 * np.dot(LT_pauli, H).trace()
    
LT_weights

array([1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [15]:
# Decompose matrix H into Pauli matrices
def decomposePauli(H):
    sigma_x = np.array(
        [
            [0, 1],  
            [ 1, 0]
        ], 
        dtype=np.complex128)
    sigma_y = np.array(
        [
            [0, -1j],
            [1j, 0]
        ], 
        dtype=np.complex128)
    sigma_z = np.array(
        [
            [1, 0],  
            [0, -1]
        ], 
        dtype=np.complex128)
    I = np.array(
        [
            [1, 0],  
            [ 0, 1]
        ], 
        dtype=np.complex128)
    S = [I, sigma_x, sigma_y, sigma_z]
    
    dim_matrix = np.shape(H)[0]
    n_qubits = int(np.log2(dim_matrix))
    
    if dim_matrix != 2**n_qubits:
        raise ValueError("matrix is not power of 2!")
        
    hilbert_space = 2**n_qubits
    n_paulis = 4**n_qubits
    
    pauli_list = np.zeros([n_paulis, n_qubits], dtype=int)
    
    for k in range(n_paulis):
        pauli_list[k,:] = numberToBase(k, 4, n_qubits)
    
    weights = np.zeros(n_paulis, dtype=np.complex128)
    
    for k in range(n_paulis):
        pauli = S[pauli_list[k][0]]

        for n in range(1,n_qubits):
            pauli = np.kron(pauli, S[pauli_list[k][n]])

        #weights[k] = 1/hilbertspace* (np.dot(H.conjugate().transpose(), pauli)).trace()
        weights[k] = 1/hilbert_space * np.dot(pauli, H).trace()

    return pauli_list, weights

In [29]:
# define Pauli operators needed for cost function and constraints of NISQ SDP
qt_dims = [
    [2 for _ in range(n_qubits)],
    [2 for _ in range(n_qubits)]
]

In [30]:
qt_dims

[[2, 2, 2], [2, 2, 2]]

In [31]:
hilbert_space = 2**n_qubits
print(hilbert_space)

8


In [32]:
constraint = np.zeros([hilbert_space, hilbert_space])
print(constraint)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [33]:
# define operator for constraints
constraints_op = []

# Lovasz-Theta number
if model == 0:
    # define matrix for cost operator
    cost_op = qt.Qobj(np.ones([hilbert_space, hilbert_space]), dims=qt_dims)
    
    for k in range(hilbert_space):
        for n in range(k+1, hilbert_space): 
            if graph[k,n] != 0:               
                constraint[n,k] = 1
                constraint[k,n] = 1
                constraints_op.append(qt.Qobj(constraint, dims=qt_dims)) # very wierd decomposition of graph
# Bell games
elif model == 1:
    J = np.array([
        [ 0.   ,  0.   ,  0.125,  0.125],
        [ 0.   ,  0.   ,  0.125, -0.125],
        [ 0.125,  0.125,  0.   ,  0.   ],
        [ 0.125, -0.125,  0.   ,  0.   ]
    ])
    cost_op = qt.Qobj(J, dims=qt_dims)
    
    for k in range(hilbert_space):
        constraint[k,k] = 1
        constraints_op.append(qt.Qobj(constraint, dims=qt_dims)) 

In [34]:
cost_op

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]


In [36]:
constraints_op

[Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
 Qobj data =
 [[0. 1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]],
 Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
 Qobj data =
 [[0. 1. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]],
 Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
 Qobj data =
 [[0. 1. 0. 0. 1. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]],
 Quantum o

In [16]:
n_constraints = len(constraints_op) # number of constraints
pauli_string_cost = []
weights_pauli_cost = []
pauli_string_constraints = []
weights_pauli_constraints = []

# get pauli strings and weights for cost function
pauli_string_cost, weights_pauli_cost = decomposePauli(cost_op)

# get pauli strings and weights for constraint
for k in range(n_constraints):
    pauli_list, weights = decomposePauli(constraints_op[k])
    pauli_string_constraints.append(pauli_list)
    weights_pauli_constraints.append(weights)

In [37]:
n_constraints

12

In [42]:
pauli_string_cost

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 0, 3],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 1, 3],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [0, 2, 3],
       [0, 3, 0],
       [0, 3, 1],
       [0, 3, 2],
       [0, 3, 3],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 0, 3],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 1, 3],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [1, 2, 3],
       [1, 3, 0],
       [1, 3, 1],
       [1, 3, 2],
       [1, 3, 3],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 0, 3],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 1, 3],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2],
       [2, 2, 3],
       [2, 3, 0],
       [2, 3, 1],
       [2, 3, 2],
       [2, 3, 3],
       [3, 0, 0],
       [3, 0, 1],
       [3, 0, 2],
       [3, 0, 3],
       [3, 1, 0],
       [3, 1, 1],
       [3, 1, 2],
       [3,

In [43]:
weights_pauli_cost

array([1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [44]:
pauli_string_constraints

[array([[0, 0, 0],
        [0, 0, 1],
        [0, 0, 2],
        [0, 0, 3],
        [0, 1, 0],
        [0, 1, 1],
        [0, 1, 2],
        [0, 1, 3],
        [0, 2, 0],
        [0, 2, 1],
        [0, 2, 2],
        [0, 2, 3],
        [0, 3, 0],
        [0, 3, 1],
        [0, 3, 2],
        [0, 3, 3],
        [1, 0, 0],
        [1, 0, 1],
        [1, 0, 2],
        [1, 0, 3],
        [1, 1, 0],
        [1, 1, 1],
        [1, 1, 2],
        [1, 1, 3],
        [1, 2, 0],
        [1, 2, 1],
        [1, 2, 2],
        [1, 2, 3],
        [1, 3, 0],
        [1, 3, 1],
        [1, 3, 2],
        [1, 3, 3],
        [2, 0, 0],
        [2, 0, 1],
        [2, 0, 2],
        [2, 0, 3],
        [2, 1, 0],
        [2, 1, 1],
        [2, 1, 2],
        [2, 1, 3],
        [2, 2, 0],
        [2, 2, 1],
        [2, 2, 2],
        [2, 2, 3],
        [2, 3, 0],
        [2, 3, 1],
        [2, 3, 2],
        [2, 3, 3],
        [3, 0, 0],
        [3, 0, 1],
        [3, 0, 2],
        [3, 0, 3],
        [3, 

In [45]:
weights_pauli_constraints

[array([0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j]),
 array([0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j,
        0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j, 0.25+0.j, 0.  +0.j,
        0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j

In [17]:
# get Pauli strings to generate ansatz space
# get all possible combinations of x Paulis
n_paulis = 2**n_qubits
ansatz_pauli = np.zeros([n_paulis, n_qubits], dtype=int)
for k in range(n_paulis):
    ansatz_pauli[k,:] = numberToBase(k, 2, n_qubits)

In [46]:
ansatz_pauli

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

In [18]:
# solve classical SDP as reference
X = cp.Variable((hilbert_space, hilbert_space), symmetric=True)

# Constraints
# The operator >> denotes matrix inequality.
constraints = [X >> 0]# positive semidefinite
    
# Lovasz-theta number
if model == 0:
    J = np.ones([hilbert_space, hilbert_space])
    
    constraints += [cp.trace(X) == 1] # trace 1 of density matrix
    
    for i in range(hilbert_space):
        for j in range(hilbert_space):
            if graph[i,j] == 1:
                constraints += [X[i,j] == 0]
# Bell game
elif model == 1:
    J = np.array([
        [ 0.   ,  0.   ,  0.125,  0.125],
        [ 0.   ,  0.   ,  0.125, -0.125],
        [ 0.125,  0.125,  0.   ,  0.   ],
        [ 0.125, -0.125,  0.   ,  0.   ]
    ])

    #constraint diagonals to be 1
    for i in range(hilbert_space):
        constraints += [X[i,i] == 1]
    
prob = cp.Problem(cp.Maximize(cp.trace(J@X)), constraints)  
theta = prob.solve(solver="CVXOPT", verbose=verbose_SDP)
    
solution_exact = qt.Qobj(X.value, dims=qt_dims)
print("\nExact SDP value:",np.trace(np.dot(J, X.value)))

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Sep 08 09:15:59 AM: Your problem has 64 variables, 26 constraints, and 0 parameters.
(CVXPY) Sep 08 09:15:59 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 08 09:15:59 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 08 09:15:59 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 08 09:15:59 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) Sep 08 09:15:59 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> C

In [19]:
# generate initial_state that is used for NISQ SDP solver
# type of ansatz  
if ansatz_state == 0: # product state all zero
    initial_state = qt.tensor([qt.basis(levels, 0) for _ in range(n_qubits)])
elif ansatz_state == 1: # randomized circuit as initial state, consists of y rotations and CNOT gates
    depth = 5 # number of layers of circuit
    rand_angles = rng.random([depth, n_qubits]) * 2 * np.pi # randomized angles
    rand_pauli = np.ones([depth, n_qubits], dtype=int) * 2 # y rotations

    entangling_gate_index = [[2*j, 2*j+1] for j in range(n_qubits//2)] + [[2*j+1, 2*j+2] for j in range((n_qubits-1)//2)]

    entangling_layer = prod([qt.qip.operations.cnot(n_qubits, j, k) for j, k in entangling_gate_index[::-1]])
    #need [::-1] to invert order so that unitaries are multiplied in correct order

    initial_state = qt.tensor([qt.basis(levels, 0) for _ in range(n_qubits)])

    ## do layered ansatz construction
    for j in range(depth):
        rot_op = []
        for k in range(n_qubits):
            angle = rand_angles[j][k]
            if(rand_pauli[j][k] == 1):
                rot_op.append(qt.qip.operations.rx(angle))
            elif(rand_pauli[j][k] == 2):
                rot_op.append(qt.qip.operations.ry(angle))
            elif(rand_pauli[j][k] == 3):
                rot_op.append(qt.qip.operations.rz(angle))

        initial_state *= qt.tensor(rot_op) * entangling_layer

In [20]:
## generate ansatz space by using initial_state and applying Pauli operators on it
expand_strings = ansatz_pauli[:n_ansatz]

n_ansatz_space = len(expand_strings)

expand_states = []
for i in range(n_ansatz_space):
    expand_states.append(get_pauli_op(expand_strings[i], opI, opX, opY, opZ) * initial_state)
    
E_matrix = np.zeros([n_ansatz_space, n_ansatz_space], dtype=np.complex128)

D_matrix = np.zeros([n_ansatz_space, n_ansatz_space], dtype=np.complex128)

constraint_matrix = np.zeros([n_constraints, n_ansatz_space, n_ansatz_space], dtype=np.complex128)


# calculate E, D and constraint matrix as overlaps of the ansatz space
# use the decomposed Pauli operators to measure the overlaps

for m in range(n_ansatz_space):
    for n in range(n_ansatz_space):
        E_matrix[m,n] = expand_states[m].overlap(expand_states[n])
        
        weights = weights_pauli_cost
        pauli_list = pauli_string_cost
        for j in range(len(pauli_list)):
            if weights[j] != 0:
                D_matrix[m,n] += weights[j] * (expand_states[m].overlap(get_pauli_op(pauli_list[j], opI, opX, opY, opZ)\
                                                                        * expand_states[n])) 
        
        for k in range(n_constraints):
            weights = weights_pauli_constraints[k]
            pauli_list = pauli_string_constraints[k]

            for j in range(len(pauli_list)):
                if weights[j] != 0:
                    constraint_matrix[k][m,n] += weights[j] * \
                    (expand_states[m].overlap(get_pauli_op(pauli_list[j], opI, opX, opY, opZ) * expand_states[n]))

In [21]:
# run classical part of NISQ SDP solver, solve SDP over measured overlap matrices
beta = cp.Variable((n_ansatz_space, n_ansatz_space), symmetric=True)

# Constraints
# The operator >> denotes matrix inequality.
constraints = [beta >> 0] #positive semidefinite

## Lovasz theta
if model == 0: 
    constraints += [cp.real(cp.trace(beta@E_matrix)) == 1] #trace 1 condition
    for k in range(n_constraints):
        if np.sum(np.abs(constraint_matrix[k])) > 0: #remove empty constraints as they can cause issues
            constraints += [cp.real(cp.trace(beta@constraint_matrix[k])) == 0]
## Bell games
elif model == 1:
    n_do_constraints = n_constraints
    if n_ansatz_space == 2:
        n_do_constraints -= 1 # remove one constraint for 2 ansatz states as there is a bug with the solver that causes issues here
    for k in range(n_do_constraints):
        if np.sum(np.abs(constraint_matrix[k])) > 0: #remove empty D_matrix as they can cause issues
            constraints += [cp.real(cp.trace(beta@constraint_matrix[k])) == 1]

# initialise sdp sovlver for NISQ problem
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(beta@D_matrix))), constraints)  
    
# run solver
theta_NISQ = prob.solve(solver="CVXOPT", verbose=verbose_SDP)
beta_NISQ = beta.value
    
print("SDP finished, cost function:", np.trace(np.dot(D_matrix, beta_NISQ)))
print("\nTrace constraint:", np.trace(np.dot(E_matrix, beta_NISQ)))
        
## hybrid density matrix as generated by NISQ SDP solver
NISQ_state = sum([sum([(expand_states[i] * expand_states[k].dag()) * beta_NISQ[i,k] for i in range(n_ansatz_space)]) for k in range(n_ansatz_space)])

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Sep 08 09:16:01 AM: Your problem has 64 variables, 14 constraints, and 0 parameters.
(CVXPY) Sep 08 09:16:01 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 08 09:16:01 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 08 09:16:01 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 08 09:16:01 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) Sep 08 09:16:01 AM: Reduction chain: Complex2Real -> FlipObjective -> Dcp2Cone -> Cvx

In [22]:
## evaluate error in regards to const function and constraints

## exact result from classical SDP for constraints and cost
exact_constraints = np.zeros(n_constraints)
exact_cost = np.real(qt.expect(cost_op, solution_exact))
for k in range(n_constraints):
    exact_constraints[k] = np.real(qt.expect(constraints_op[k], solution_exact))

## from NISQ SDP solver
NISQ_constraints = np.zeros(n_constraints)
NISQ_cost = np.real(qt.expect(cost_op, NISQ_state))
for k in range(n_constraints):
    NISQ_constraints[k] = np.real(qt.expect(constraints_op[k], NISQ_state))

# error between exact and NISQ solution
error_cost = np.abs(NISQ_cost-exact_cost)
error_constraints = np.sum([np.abs(NISQ_constraints[k]-exact_constraints[k]) for k in range(n_constraints)])

In [23]:
print("Total error between exact and NISQ SDP solution:", error_cost + error_constraints)
print("Error cost:", error_cost)
print("Error constraints:", error_constraints)

Total error between exact and NISQ SDP solution: 0.0
Error cost: 0.0
Error constraints: 0.0
